# **CONFIGURATION** de PySpark

## Configuration des variables d'environnement

## Vérification des variables d'environnement

In [5]:
import os
print(os.environ['SPARK_HOME'])
print(os.environ['PATH'])
print(os.environ['JAVA_HOME'])

/home/christophe/spark/spark-2.4.4-bin-hadoop2.7
/usr/bin:/home/christophe/spark/spark-2.4.4-bin-hadoop2.7/bin:/home/christophe/anaconda3/envs/DS_projet8/bin:/home/christophe/anaconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin
/usr


## Importation des librairies

In [1]:
# Import libraries
from pyspark.sql import SparkSession

In [2]:
import pyspark
pyspark.__version__

'2.4.4'

## Instanciation de l'objet *SparkSession*

In [2]:
# Creating SparkSession
spark = (SparkSession
            .builder
            .master("local[*]")
            .appName("Projet 8")
            .config('spark.jars.packages', 'databricks:spark-deep-learning:1.5.0-spark2.4-s_2.11')
            .getOrCreate()
)

In [ ]:
# Installation d'un package Maven

In [ ]:
!mvn install:install-file -Dfile=/home/christophe/Insync/lebrun_christo@yahoo.fr/OneDrive/OpenClassrooms/parcoursDS/projet8/s3/libs/spark-deep-learning-1.5.0-spark2.4-s_2.11.jar -DgroupId=databricks \
        -DartifactId=spark-deep-learning -Dversion=1.5.0-spark2.4-s_2.11 -Dpackaging=jar

# Essai d'installation de dépendance locale

In [ ]:
# Closing the sparkSession
spark.stop()

In [ ]:
import os

# load pyspark with databricks package
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/christophe/Desktop/Link\ to\ projet8/s3/libs/spark-deep-learning-1.5.0-spark2.4-s_2.11.jar pyspark-shell'

In [ ]:
# Creating SparkSession
spark = (SparkSession
            .builder
            .master("local[*]")
            .appName("Projet 8")
            .config('spark.jars', "/home/christophe/Desktop/Link\ to\ projet8/s3/libs/spark-deep-learning-1.5.0-spark2.4-s_2.11.jar")
            .getOrCreate()
)

In [ ]:
# Creating SparkSession
spark = (SparkSession
            .builder
            .master("local[*]")
            .appName("Projet 8")
            .config('spark.jars', "/home/christophe/Insync/lebrun_christo@yahoo.fr/OneDrive/OpenClassrooms/parcoursDS/projet8/s3/libs/spark-deep-learning-1.5.0-spark2.4-s_2.11.jar")
            .getOrCreate()
)

In [4]:
from sparkdl import DeepImageFeaturizer

Using TensorFlow backend.


# **EXTRACTION** des données

## Suppression des espaces dans les noms de répertoires

In [11]:
# Initializations
path = './data/'

# Parsing the path
for root, dirs, files in os.walk(path):
    
    # Checking if last level of directory
    if dirs==[]:
        
        # Get the name of the directory
        old_dir = root.split("/")[-1]
        
        # Checking if the name contains blank spaces
        if " " in old_dir:

            # Get the name of the base
            base = root.strip(old_dir)
            # Create new name for directory
            new_dir = old_dir.replace(" ", "_")

            print("Rename:", old_dir, "to:", new_dir)
            os.rename(os.path.join(base, old_dir), os.path.join(base, new_dir))

## Chargement des images et association des labels

In [16]:
## Parsing directory
import os
from pyspark.sql.functions import lit

# Initializations
path = './s3/data/Test/'
i = 0

# Parsing the directories and files in the path
for root, dirs, files in os.walk(path):
    
    # If no sub-directories (limit to 10 categories for exploration)
    if dirs==[] and i<10:
        
        # Get the name of the last directory
        category = root.split("/")[-1]
        print(category)

        # Read image data using new image scheme
        cat_df = (spark
                .read
                .format("image")
                .load(root)
        )
        
        # Displays a few rows
        # cat_df.show(3)
        
        # Add a column with label
        cat_df = cat_df.withColumn(
            'label',
            lit(category),
        )
        
        # Contatenate the data
        try:
            image_df = image_df.unionAll(cat_df)
        except NameError:
            image_df = cat_df
        
        # Increment counter (for exploration)
        i += 1

print("Loading completed.")

Pear_Kaiser
Cocos
Mango_Red
Potato_Sweet
Pomelo_Sweetie
Limes
Huckleberry
Clementine
Potato_White
Nectarine
Loading completed.


In [17]:
image_df.limit(3).show()

+--------------------+-----------+
|               image|      label|
+--------------------+-----------+
|[file:///home/chr...|Pear_Kaiser|
|[file:///home/chr...|Pear_Kaiser|
|[file:///home/chr...|Pear_Kaiser|
+--------------------+-----------+



In [20]:
image_df = image_df.limit(5)

In [21]:
image_df = image_df.show(10)

+--------------------+-----------+
|               image|      label|
+--------------------+-----------+
|[file:///home/chr...|Pear_Kaiser|
|[file:///home/chr...|Pear_Kaiser|
|[file:///home/chr...|Pear_Kaiser|
|[file:///home/chr...|Pear_Kaiser|
|[file:///home/chr...|Pear_Kaiser|
+--------------------+-----------+



In [15]:
del image_df

In [ ]:
    ##################################
    # Loading images
    ##################################

    ROOT_PATH = sys.argv[1]

    # RUN MODE
    DATA_PATH = os.path.join(ROOT_PATH, "data/")

    # TEST MODE
    DATA_PATH = os.path.join(ROOT_PATH, "data/Test/Apricot/") # REMOVE the line for production

    image_df = spark.read.format("image").load(DATA_PATH)

In [ ]:
from pyspark.sql.functions import split, element_at

# Getting the label by splitting the path of the image and getting its last directory
image_df = (image_df
            .withColumn(
                'label',
                element_at(
                    split(
                        image_df['image.origin'],
                        "/"),
                    -2))
            .show(2))

## Exploration

In [ ]:
img_count = image_df.count()
print("Total number of images:", img_count)

In [ ]:
print("Total number of categories:")
image_df.select('label').distinct().count()

In [ ]:
image_df.printSchema()

In [ ]:
# Acceding to data of images
nb_height = image_df.select("image.height").distinct().count()
print("Nombre de hauteurs différentes:", nb_height)

nb_width = image_df.select("image.width").distinct().count()
print("Nombre de largeurs différentes:", nb_width)

# **TRAITEMENT** des données

## Importations des fonctions

In [ ]:
from sparkdl import DeepImageFeaturizer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

## Extraction de caractéristiques par CNN

In [ ]:
featurizer = DeepImageFeaturizer(
    inputCol="image",
    outputCol="features",
    modelName="ResNet50"
)

In [ ]:
!pip install sparkdl --upgrade
!pip show sparkdl

In [ ]:
# Build our model
features_df = featurizer.transform(image_df)

In [ ]:
features_df.printSchema()

In [ ]:
# for debug purpose…
features_df = image_df

## Configuration de Boto3

In [ ]:
import config.p8_fruits_credentials as credentials

# Get the flickr_key and flickr_secret
ACCESS_KEY = credentials.ACCESS_KEY
SECRET_KEY = credentials.SECRET_KEY

In [ ]:
import boto3

s3_resource = boto3.resource(
    's3',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY,
    region_name='eu-west-3',
)

my_bucket = s3_resource.Bucket('p8-fruits')

## Téléversement sur S3

In [ ]:
# Save a file on the S3 bucket in the 'results' directory
my_bucket.put_object(
    Body=str(img_count), # content of the file to save
    Key='results/hello.txt', # path of the new_file on the bucket
)

In [ ]:
# Download the file back and read it
my_bucket.download_file(Key='results/hello.txt', Filename='hello_back.txt')
open('hello_back.txt').read()

In [ ]:
# Download the file back and read it
my_bucket.download_file(Key='results/features.txt', Filename='features.txt')
open('features.txt').read()

## Physical plans

In [ ]:
image_df.explain()

# **EXTINCTION** de la session Spark

In [ ]:
# Closing the sparkSession
spark.stop()

# **LECTURE** des résultas (format *parquet*)

In [4]:
results_df = spark.read.format("parquet").load("s3/results/Dec-20-2019 09:25:49")

In [5]:
results_df.count()

300

In [6]:
results_df.show(5)

+--------------------+--------------+--------------------+
|            features|         label|              origin|
+--------------------+--------------+--------------------+
|[2.20138478279113...|     Raspberry|file:/home/christ...|
|[2.03587174415588...|     Raspberry|file:/home/christ...|
|[0.29767277836799...|Pineapple_Mini|file:/home/christ...|
|[2.59762907028198...|     Raspberry|file:/home/christ...|
|[2.10859084129333...|     Raspberry|file:/home/christ...|
+--------------------+--------------+--------------------+
only showing top 5 rows



In [7]:
results_df.limit(2)

DataFrame[features: vector, label: string, origin: string]

In [9]:
results_df.show(5)

+--------------------+--------------+--------------------+
|            features|         label|              origin|
+--------------------+--------------+--------------------+
|[2.20138478279113...|     Raspberry|file:/home/christ...|
|[2.03587174415588...|     Raspberry|file:/home/christ...|
|[0.29767277836799...|Pineapple_Mini|file:/home/christ...|
|[2.59762907028198...|     Raspberry|file:/home/christ...|
|[2.10859084129333...|     Raspberry|file:/home/christ...|
+--------------------+--------------+--------------------+
only showing top 5 rows



# **QUESTIONS / PROBLÈMES TECHNIQUES**


* Plusieurs options pour la gestion de la mémoire :
    * executor-cores
    * nombre de partitions (dans le code du DF/RDD)
    * -executor-memory 10g
    * --driver-memory 10g
    
    
* Options pour les notebooks
    * Notebook EMR
        * pb : configurer une SparkSession déjà initiée
        * pb : donner l'accès à Internet (vérifier avec Ping)
        * copier le .jar sur le master
    * Jupyter sur le master node
    * Zeppelin
    * DataBricks
    * Google collabs
    * SageMaker notebooks
    * …
 
 
* Logiciels sur clusters
    * Spark
    * Livy (pour notebooks)
    * Hadoop
    * Ganglia
    * Zeppelin

# TODO

* Rendre public les fichiers de données et les résultats sur S3 ? Le notebook ?
* Voir le besoin d'installer les modules (sys, os) sur les nodes ou le master avant l'exécution
* Importer .jar avec notebook EMR… Inclure le paquet local comme dépendence Maven ?
    * https://docs.amazonaws.cn/en_us/emr/latest/ManagementGuide/emr-managed-notebooks-considerations.html
    * https://stackoverflow.com/questions/57473914/adding-external-jars-in-emr-notebooks
* Prétraitements : 
    * https://databricks.com/blog/2018/12/10/introducing-built-in-image-data-source-in-apache-spark-2-4.html
* 1'000 images, est-ce suffisant ? 1 heure de travail, sur 1 PC… Ou essai sur 2 PC ?
* Split des labels : "Apple Golden 1"
* Différence entre mode cluster et mode client ?